In [2]:
import importlib
from transformer import Transformer # this is the transformer.py file
import transformer
import torch
print(transformer.__file__)

import numpy as np

C:\Users\Lenovo\Desktop\Transformer\Code run\transformer.py


In [3]:

english_file = 'C:\\Users\\Lenovo\\Desktop\\Transformer\\Code run\\train.en' 
hindi_file = 'C:\\Users\\Lenovo\\Desktop\\Transformer\\Code run\\train.hi' 

START_TOKEN = ''
PADDING_TOKEN = ''
END_TOKEN = ''


hindi_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', 'ˌ', 
                    'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ॠ', 'ऌ', 'ए', 'ऐ', 'ओ', 'औ', 
                    'क', 'ख', 'ग', 'घ', 'ङ', 
                    'च', 'छ', 'ज', 'झ', 'ञ', 
                    'ट', 'ठ', 'ड', 'ढ', 'ण', 
                    'त', 'थ', 'द', 'ध', 'न', 
                    'प', 'फ', 'ब', 'भ', 'म', 
                    'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', 
                    '़', 'ऽ', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॄ', 'े', 'ै', 'ो', 'ौ', '्', '॑', '॒', '॓', '॔', 
                    '०', '१', '२', '३', '४', '५', '६', '७', '८', '९', PADDING_TOKEN, END_TOKEN]

english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`', 
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 
                        'y', 'z', 
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]
     


In [4]:
index_to_hindi = {k:v for k,v in enumerate(hindi_vocabulary)}
hindi_to_index = {v:k for k,v in enumerate(hindi_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [5]:
with open(english_file, 'r', encoding='utf-8') as file:
    english_sentences = file.readlines()
with open(hindi_file, 'r', encoding='utf-8') as file:
    hindi_sentences = file.readlines()

# Limit Number of sentences
TOTAL_SENTENCES = 200000
english_sentences = english_sentences[:TOTAL_SENTENCES]
hindi_sentences = hindi_sentences[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
hindi_sentences = [sentence.rstrip('\n') for sentence in hindi_sentences]

In [6]:
english_sentences[:10]


['in reply, pakistan got off to a solid start.',
 'the european union has seven principal decision-making bodies, its institutions: the european parliament, the european council, the council of the european union, the european commission, the court of justice of the european union, the european central bank and the european court of auditors.',
 'the congress leader represents sivaganga lok sabha segment from tamil nadu.',
 'prompt the user about connection attempts',
 'further, the minister announced that deposit insurance and credit guarantee corporation (dicgc) has been permitted to increase deposit insurance coverage to rs.',
 'therefore, brothers, be more diligent to make your calling and election sure. for if you do these things, you will never stumble.',
 'the review committee meeting chaired by the district judges will be attended by collectors, sps, superintendents of jails and secretary of district legal services authority.',
 'police is present on the spot.',
 'prime ministe

In [7]:
hindi_sentences[:10]

['जिसके जवाब में पाक ने अच्छी शुरुआत की थी.',
 'यूरोपीय संघ के महत्वपूर्ण संस्थानों में यूरोपियन कमीशन, यूरोपीय संसद, यूरोपीय संघ परिषद, यूरोपीय न्यायलय एवं यूरोपियन सेंट्रल बैंक इत्यादि शामिल हैं।',
 'कांग्रेस नेता तमिलनाडु से शिवगंगा लोकसभा क्षेत्र का प्रतिनिधित्व करते हैं.',
 'संबंधन प्रयास के बारे में उपयोक्ता को प्रांप्ट करें',
 'वित्त मंत्री ने घोषणा कि जमा बीमा और ऋण गारंटी निगम (डीआईसीजीसी) को जमा राशि बीमा का दायरा, जो इस समय 1 लाख रुपये है उसे बढ़ाकर प्रति जमाकर्ता 5 लाख रुपये करने की अनुमति प्रदान कर दी गई है।',
 'इस कारण हे भाइयों, अपने बुलाए जाने, और चुन लिये जाने को सिद्ध करने का भली भांति यत्न करते जाओ, क्योंकि यदि ऐसा करोगे, तो कभी भी ठोकर न खाओगे।',
 'समिति में जिला एवं सत्र न्यायाधीश, कलेक्टर, पुलिस अधीक्षक व जिला विधिक सेवा प्राधिकरण के सचिव शामिल रहते हैं।',
 'पुलिस मौके पर मौजूद है।',
 'जम्मू एवं कश्मीर विधानसभा के लिए दूसरे चरण के तहत होने वाले मतदान के लिए प्रधानमंत्री नरेंद्र मोदी भारतीय जनता पार्टी (भाजपा) के पक्ष में शुक्रवार को यहां रैलियां करेंगे।',
 'और 305

In [8]:
import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length hindi: {np.percentile([len(x) for x in hindi_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )

97th percentile length hindi: 260.0
97th percentile length English: 267.0


In [9]:

max_sequence_length = 200

def is_valid_tokens(sentence, vocab):
    #print(sentence)
    for token in list(set(sentence)):
        #print(token)
        if token not in vocab:
            
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(hindi_sentences)):
    hindi_sentence, english_sentence = hindi_sentences[index], english_sentences[index]
    if is_valid_length(hindi_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(hindi_sentence, hindi_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(hindi_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 200000
Number of valid sentences: 51965


In [10]:

hindi_sentences = [hindi_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [11]:
hindi_sentences[:3]

['जिसके जवाब में पाक ने अच्छी शुरुआत की थी.',
 'कांग्रेस नेता तमिलनाडु से शिवगंगा लोकसभा क्षेत्र का प्रतिनिधित्व करते हैं.',
 'संबंधन प्रयास के बारे में उपयोक्ता को प्रांप्ट करें']

In [12]:

import torch

d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 200
hi_vocab_size = len(hindi_vocabulary)

transformer = Transformer(d_model, 
                          ffn_hidden,
                          num_heads, 
                          drop_prob, 
                          num_layers, 
                          max_sequence_length,
                          hi_vocab_size,
                          english_to_index,
                          hindi_to_index,
                          START_TOKEN, 
                          END_TOKEN, 
                          PADDING_TOKEN)

1 => transformer


In [13]:

transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(71, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding):

In [14]:

from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, hindi_sentences):
        self.english_sentences = english_sentences
        self.hindi_sentences = hindi_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.hindi_sentences[idx]

In [15]:

dataset = TextDataset(english_sentences, hindi_sentences)

In [16]:
len(dataset)


51965

In [17]:
dataset[1]

('the congress leader represents sivaganga lok sabha segment from tamil nadu.',
 'कांग्रेस नेता तमिलनाडु से शिवगंगा लोकसभा क्षेत्र का प्रतिनिधित्व करते हैं.')

In [18]:

train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [19]:

for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

[('in reply, pakistan got off to a solid start.', 'the congress leader represents sivaganga lok sabha segment from tamil nadu.', 'prompt the user about connection attempts', 'all 176 passengers died in the incident.', 'not invited', 'coconut oil and lemon juice pack:', 'new delhi, 14th may, 2010', 'algeria military plane crash', 'up and down', 'probably a lot of you know the story of the two salesmen who went down to africa in the 1900s.', 'tejashwi slams nitish government', 'zenan, and hadashah, and migdal-gad,', 'but all was not over.', 'what is jcpoa?', 'this, in a way, is one of the challenges of how to interpret a really good idea.', 'director: shyam benegal', 'obtain loans from a bank or an individual.', 'are doing.', 'they should stay away from the technology and spend more time studying.', 'but those who disbelieve in the everlasting life deviate from the path.', 'he is at present lodged in a prison in london.', 'in both conditions your doctor can prescribe you some analgesics 

In [20]:

from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=hindi_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [21]:

NEG_INFTY = -1e9

def create_masks(eng_batch, hi_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
        
        eng_sentence_length, hi_sentence_length = len(eng_batch[idx]), len(hi_batch[idx])
        eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
        hi_chars_to_padding_mask = np.arange(hi_sentence_length + 1, max_sequence_length)
        encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
        encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
        decoder_padding_mask_self_attention[idx, :, hi_chars_to_padding_mask] = True
        decoder_padding_mask_self_attention[idx, hi_chars_to_padding_mask, :] = True
        decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
        decoder_padding_mask_cross_attention[idx, hi_chars_to_padding_mask, :] = True
    
    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [22]:
max(len(sublist) for sublist in hindi_sentences)

198

In [23]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    #print("1iterator Done")
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        #print("2iterator done")
        #print(batch_num)
        eng_batch, hi_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, hi_batch)
        optim.zero_grad()
        hi_predictions = transformer(eng_batch,
                                     hi_batch,
                                     encoder_self_attention_mask.to(device), 
                                     decoder_self_attention_mask.to(device), 
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        #print("3iterator done")
        labels = transformer.decoder.sentence_embedding.batch_tokenize(hi_batch, start_token=False, end_token=True)
        loss = criterian(
            hi_predictions.view(-1, hi_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == hindi_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            #print("4 sucess")
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Hindi Translation: {hi_batch[0]}")
            hi_sentence_predicted = torch.argmax(hi_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in hi_sentence_predicted:
                
                if idx == hindi_to_index[END_TOKEN]:
                    
                    break
                predicted_sentence += index_to_hindi[idx.item()]
            print(f"hindi Prediction: {predicted_sentence}")


            transformer.eval()
            hi_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, hi_sentence)
                predictions = transformer(eng_sentence,
                                          hi_sentence,
                                          encoder_self_attention_mask.to(device), 
                                          decoder_self_attention_mask.to(device), 
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_hindi[next_token_index]
                hi_sentence = (hi_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                    break
            
            print(f"Evaluation translation (should we go to the mall?) : {hi_sentence}")
            print("-------------------------------------------")

Epoch 0
Iteration 0 : 5.749130725860596
English: in reply, pakistan got off to a solid start.
Hindi Translation: जिसके जवाब में पाक ने अच्छी शुरुआत की थी.
hindi Prediction: 
Evaluation translation (should we go to the mall?) : ('र ६६६   रररररर6                न     डन      नन                नचचच                   ननन     च६६नन      ाानननननननन',)
-------------------------------------------
Iteration 100 : 3.40419864654541
English: jharkhand will vote in five phases.
Hindi Translation: झारखंड में पांच चरणों में हुए मतदान
hindi Prediction: कस्                                                                                                      ा              ा                                  रकक          र                           र   
Evaluation translation (should we go to the mall?) : ('रर                                                                                                                                                                                                      '

Evaluation translation (should we go to the mall?) : ('हैं को के के के हैं हैं है. है..........................................................................................................................................................................',)
-------------------------------------------
Iteration 1300 : 2.6631741523742676
English: the vigilance awareness week, 2020 will be observed from tuesday till 2nd of november.
Hindi Translation: 28 अक्टूबर से 2 नवंबर के बीच सतर्कता जागरूकता सप्ताह मनाया जा रहा है.
hindi Prediction: इ0 हपेर्र  के क0केि ा के का ाकेाीरा  का ा     क््रा  केे ा का क ै कै.         र ररर  र  र    ररर  र      र       रररर ररर र    रर   र           ररर र    रर    र रर र   रर  र र र     र       र ररर     
Evaluation translation (should we go to the mall?) : ('क्या का के के का है. है.................................................................................................................................................................................',

Iteration 600 : 2.4522485733032227
English: natures gift
Hindi Translation: प्रकृति की देन
hindi Prediction: क्राात यका का ेरररर ररर ररररटरररररररररररररररररररररररररररररररररटटररररररररररररटररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररशरररररशरररशररररररररररर रररररररररररररर
Evaluation translation (should we go to the mall?) : ('हम हैं को कर हैं है?"""""""""""""""""""""""""""""""""""""""""""""""""""""?????????????????????.""""""""""""""""""""""""""""""""""""""""""""""""?????...................?????????.""""???????............',)
-------------------------------------------
Iteration 700 : 2.4607741832733154
English: there is no negligence.
Hindi Translation: किसी प्रकार की लापरवाही नहीं की जा रही है.
hindi Prediction: ह्स  सरराार के सिरन ा   हहीं.हर हारहही.है..ररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररररर
Evaluation translation (should we go t

Evaluation translation (should we go to the mall?) : ('क्या क्या क्या क्या है?"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""???""""""""""""""""""???????',)
-------------------------------------------
Epoch 2
Iteration 0 : 2.3047749996185303
English: in reply, pakistan got off to a solid start.
Hindi Translation: जिसके जवाब में पाक ने अच्छी शुरुआत की थी.
hindi Prediction: इिसेे काा  कें करर से कप्म  काल्क  हर हा "  रर र  र रर   रर   र रररररर 1रररररर   रजररररररररररररर11रररटररवररपररररररररररररररजररररररररररररसरररररररररररररर रसरररजजरजरर ररजजश   र  जरररपररर  स रररहरहररपररमहज
Evaluation translation (should we go to the mall?) : ('क्या क्या क्या क्या क्या है?"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""?????',)
-------------------------------------------
Iteration 100 : 2.2

Evaluation translation (should we go to the mall?) : ('क्या क्या क्या क्या क्या है? है? है??""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""हे?"""""""""""""""""""""?????',)
-------------------------------------------
Iteration 1200 : 2.237567663192749
English: two pak boats seized by bsf
Hindi Translation: बीएसएफ के दो पाकिस्तानी नावों को पकड़ा
hindi Prediction: काडम   का किनकररा  ता   कहतां कर हरऱा रररररजरररररररर रररररकररररररररररररजररररररपर रररजजरररररजरजजररररजजशररजशरजसजसरररजररररररररररररसजजररररररररररररररजरररजजरजजरजजजजररजरजजजजरवरजशजजजजरपरपरपररजगजहरररजजररजजरजज
Evaluation translation (should we go to the mall?) : ('क्या क्या क्या क्या है? है? है? है? है?""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""है?""""""""""""""""""""""""',)
-------------------------------------------
Iteration 1300 : 2.24143123626709
English: th

...

Iteration 300 : 1.8601504564285278
English: irans economy has already been going through a tough phase with severe economic sanctions imposed by the united states.
Hindi Translation: अमेरिका के कड़े आर्थिक प्रतिबंधों से ईरान की अर्थव्यवस्था की हालत पहले से ही बहुत खराब है.
hindi Prediction: इपेरिकी के पह़ा करोयित कररधिनंधिं के कमान के सन्जनाय ा्थ  के सैथ  करुे के पो कनुत हाा  कुं पगससहससससहपशससपपपससससससपपसहहहससहपगससपसहसगसपहसशहहहपपपसससगसससहसससससससहहपपहहहहजसशशगसहहहससपगससससपसपहगपससससससससससस
Evaluation translation (should we go to the mall?) : ('क्या कहा कि क्या है? क्या है? क्या है? क्या है?? है?"""""""""हा है?"""""""""क्या है?"""""""क्या है?"""""""हा है?""""""""हे हैं?"""""""""हे हैं?""""........................."?""""""""""""है?"""""""""""',)
-------------------------------------------
Iteration 400 : 1.7236621379852295
English: employees' money in up invested in mirchi-linked firm of wadhawans
Hindi Translation: उप्र के कर्मचारियों के पैसों को मिर्ची से जुड़ी वधावन की कंपनी में लगाया ग

Evaluation translation (should we go to the mall?) : ('क्या कहाँ क्या क्या है? क्या है???????"""""""क्या है????""" क्या है??"""" क्या है?""""""" क्या है?""""" क्या है?""""""क्या है?""".......................................................................',)
-------------------------------------------
Iteration 1500 : 1.8084675073623657
English: who shall be my inheritor and the inheritor of the house of jacob. and make him, my lord, well-pleasing.'
Hindi Translation: जो मेरी और याकूब की नस्ल की मीरास का मालिक हो ऐ मेरे परवरदिगार और उसको अपना पसन्दीदा बन्दा बना
hindi Prediction: औब सैंे कर जह िल के मिूतिके सौड   के हुन य को जसजुंे मह ादिगार कर उसके बपनी करंीदे   कनादे कनायजहससपजशजगसपगसगजसगससससहहहहगगगगपजगगपजपगगजगगसगजपपपधपपसससपहजजजजसपसहहजसहहहहहहहगजगहजजसपपगगपगगससससपससहसससससससससज
Evaluation translation (should we go to the mall?) : ('क्या क्या क्या क्या है? क्या है????????"""क्या है????????????????????"".......................................................................................

Evaluation translation (should we go to the mall?) : ('क्या क्या क्या है? क्या है???? क्या है???"""क्या है????""""क्या है??"""" क्या है??""""क्यों क्या है?"""" क्या है?""""""क्या?"""क्या है?"""""""..........................................................',)
-------------------------------------------
Iteration 900 : 1.7839733362197876
English: committee of councillors
Hindi Translation: पार्षदों की समिति
hindi Prediction: प्क्ट ां को स््ति ससससपगपपपपपधपपपगपसससपसपपसससपसससगपससससगसपसपसपगसगसससससससगससपपससससहसससससससससगससससससगसससगसपसपससससससपसससपसपगससससससससगसगगससगसगसगसपपसससससपसससससससससससससगसससगससगससगसससगगगगससगससससससगससससससगसगस
Evaluation translation (should we go to the mall?) : ('क्या कहां किस क्या है? क्या है??? है????""""क्या है?????................................................................................................................................................',)
-------------------------------------------
Iteration 1000 : 1.7220145463943481
English: this could spell trouble

Iteration 200 : 1.7203810214996338
English: im the first girl in my village to go to a university.
Hindi Translation: मैं अपनी जनजाति की पहली लड़की हूं जो यूनिवर्सिटी में पढ़ने आई है.
hindi Prediction: उैं सपने कातर   कर साले हो़कर जैं का कान का्त त  कें कऱाे कए है..ससससगसससगससससससससससससगपससगगससगसससससससगसगसगसपससससपपसससगसपसगगसगसगगगगगसससगगगससगगसससगगससससससससससगसगगसगसगसगगगगगगगसगगगगगगगसगसगगगगगसससससगसगसग
Evaluation translation (should we go to the mall?) : ('क्या कहां किस क्या है? क्या है? क्या है? ? रहा है? ? रहा है?? क्या है?? क्या है? ? क्या है? ? क्या है? ? क्या है???????""...............................................................................',)
-------------------------------------------
Iteration 300 : 1.7654393911361694
English: irans economy has already been going through a tough phase with severe economic sanctions imposed by the united states.
Hindi Translation: अमेरिका के कड़े आर्थिक प्रतिबंधों से ईरान की अर्थव्यवस्था की हालत पहले से ही बहुत खराब है.
hindi Prediction: इप

Evaluation translation (should we go to the mall?) : ('क्या कहां कि क्या है? क्या है?????????? क्या है?????????? क्या है??????? क्या है?? क्या है????..........................................................................................................',)
-------------------------------------------
Iteration 1400 : 1.66632080078125
English: in the past 24 hours, 1,349 new cases and 89 new fatalities were confirmed
Hindi Translation: बीते चौबीस घंटों के दौरान यहां 49,502 नए मामले सामने आए हैं और 1,058 लोगों की मौत हुई है
hindi Prediction: इिजेंहरही ीकंटों की सौरान काां क, 94  के सुमले केथने कए कैं कर 1491  कागों को सौत कोई हैंससटससहटसससगशहसससससससहसहसससहसहहहगगसगसकसटससगगगगगहसगसगहजससससससससससससससहससससससगसहसगगगगटहसगसगसगगससससससगगगगसससससससगगग
Evaluation translation (should we go to the mall?) : ('क्या कहाँ किसी को क्या है? ? क्या है???? क्या है??????? क्या है?????.................................................................................................................................

In [26]:
transformer.eval()
def translate(eng_sentence):
    eng_sentence = (eng_sentence,)
    hi_sentence = ("",)
    for word_counter in range(max_sequence_length):
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, hi_sentence)
        predictions = transformer(eng_sentence,
                              hi_sentence,
                              encoder_self_attention_mask.to(device), 
                              decoder_self_attention_mask.to(device), 
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
        next_token_prob_distribution = predictions[0][word_counter]
        next_token_index = torch.argmax(next_token_prob_distribution).item()
        next_token = index_to_hindi[next_token_index]
        hi_sentence = (hi_sentence[0] + next_token, )
        if next_token == END_TOKEN:
            break
    return hi_sentence[0]

In [27]:
translation = translate("what should we do when the day starts?")
print(translation)

क्या हम क्या करने के लिए क्या है? क्या है? ?? क्या है? ? क्या है?? क्या है????? क्या है?? क्या है?? क्या है???????? है? क्या है?? क्या है?? क्या है? क्या है?"".........................................


In [43]:
translation = translate("the sun is shining in the sky.")
print(translation)

यह सब के लिए का कि यह कर रहे हैं........................................................................................................................................................................


In [44]:
translation = translate("i like to read books every evening.")
print(translation)

मैं कि मैं कि मैं कि मैं किया गया. है...................................................................................................................................................................


In [45]:
translation = translate("the cat is sleeping under the table.")
print(translation)

इस मामले में से बाद है..................................................................................................................................................................................


In [46]:
translation = translate("we are going to the park today.")
print(translation)

हम करने के लिए को समझ सकते हैं..........................................................................................................................................................................
